# Dynamic Parking Pricing Model using Real-Time Data

Urban parking is a scarce resource—either overused (leading to queues and frustration) or underutilized (causing loss of revenue). Static pricing fails to address this. To fix this, cities are moving toward dynamic pricing, where parking fees change throughout the day depending on demand, traffic, vehicle type, and more.

This hackathon simulates that environment, challenging you to design an intelligent, real-time pricing engine that adapts to changing urban conditions for 14 parking spaces.


In [1]:
!pip install pathway bokeh --quiet

In [52]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
from pathway.internals.dtype import DATE_TIME_NAIVE
import bokeh.plotting
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource
import panel as pn
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("./data/dataset.csv", index_col=0)
df.head(3)

,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
ID,,,,,,,,,,,
0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00


In [9]:
print(df.duplicated().sum())  # No duplicates!

24


In [ ]:
print(df.isnull().sum())  # No missing values
print(df.shape)
df.drop_duplicates()  # No duplicates!

SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64
(18368, 11)


,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
ID,,,,,,,,,,,
0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...
18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18368 entries, 0 to 18367
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SystemCodeNumber        18368 non-null  object 
 1   Capacity                18368 non-null  int64  
 2   Latitude                18368 non-null  float64
 3   Longitude               18368 non-null  float64
 4   Occupancy               18368 non-null  int64  
 5   VehicleType             18368 non-null  object 
 6   TrafficConditionNearby  18368 non-null  object 
 7   QueueLength             18368 non-null  int64  
 8   IsSpecialDay            18368 non-null  int64  
 9   LastUpdatedDate         18368 non-null  object 
 10  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.7+ MB


In [ ]:
print(df["VehicleType"].unique())
print(df["SystemCodeNumber"].unique())
print(df["Capacity"].unique())
print(df["TrafficConditionNearby"].unique())

['car' 'bike' 'truck' 'cycle']
['BHMBCCMKT01' 'BHMBCCTHL01' 'BHMEURBRD01' 'BHMMBMMBX01' 'BHMNCPHST01'
 'BHMNCPNST01' 'Broad Street' 'Others-CCCPS105a' 'Others-CCCPS119a'
 'Others-CCCPS135a' 'Others-CCCPS202' 'Others-CCCPS8' 'Others-CCCPS98'
 'Shopping']
[ 577  387  470  687 1200  485  690 2009 2803 3883 2937 1322 3103 1920]
['low' 'high' 'average']


In [ ]:
df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S"
)
df = df.drop(["LastUpdatedDate", "LastUpdatedTime"], axis=1)

In [14]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [15]:
def min_max_normalize(series, eps=1e-6):
    return (series - series.min()) / (series.max() - series.min() + eps)

In [17]:
def feature_engineering(df):
    enhanced_df = df.copy()
    # ---- Normalize occupancy ----
    enhanced_df["OccupancyRatio"] = enhanced_df["Occupancy"] / enhanced_df["Capacity"]
    # Importance: Captures real-time demand pressure on parking space.
    # ---- Encode vehicle type ----
    vehicle_map = {"cycle": 1, "bike": 2, "car": 3, "truck": 4}
    enhanced_df["VehicleType"] = enhanced_df["VehicleType"].map(vehicle_map)
    # Importance: Helps model capacity consumption or pricing based on vehicle type.
    # ---- Encode traffic condition ----
    traffic_map = {"low": 0, "average": 1, "high": 2}
    enhanced_df["TrafficConditionNearby"] = enhanced_df["TrafficConditionNearby"].map(
        traffic_map
    )
    # Importance: Higher traffic may correlate with higher parking demand or queuing.
    # ---- Parse datetime and generate time-based features ----
    enhanced_df["hour"] = enhanced_df["Timestamp"].dt.hour
    enhanced_df["day_of_week"] = enhanced_df["Timestamp"].dt.dayofweek
    enhanced_df["is_weekend"] = (enhanced_df["day_of_week"] >= 5).astype(int)
    # Importance: Parking behavior changes hourly and on weekends — critical for dynamic pricing.
    # ---- Compute distance to nearest parking lot ----
    unique_lots = enhanced_df["SystemCodeNumber"].unique()
    lot_coords = {
        lot: (
            enhanced_df[enhanced_df["SystemCodeNumber"] == lot].iloc[0]["Latitude"],
            enhanced_df[enhanced_df["SystemCodeNumber"] == lot].iloc[0]["Longitude"],
        )
        for lot in unique_lots
    }
    closest_distances = []
    for _, row in enhanced_df.iterrows():
        lat1, lon1 = row["Latitude"], row["Longitude"]
        lot = row["SystemCodeNumber"]
        distances = [
            haversine(lat1, lon1, lat2, lon2)
            for other_lot, (lat2, lon2) in lot_coords.items()
            if other_lot != lot
        ]
        closest_distances.append(min(distances) if distances else 0.0)
    enhanced_df["closest_lot_distance"] = closest_distances
    # Importance: Competitive pricing and spillover effects often depend on proximity.
    # ---- Sort and compute basic rolling occupancy (3h) ----
    enhanced_df = enhanced_df.sort_values(["SystemCodeNumber", "Timestamp"])
    enhanced_df["occupancy_rolling_3h"] = enhanced_df.groupby("SystemCodeNumber")[
        "OccupancyRatio"
    ].transform(lambda x: x.rolling(3, min_periods=1).mean())
    # Importance: Smooths short-term fluctuations in occupancy, captures micro-trends.
    # ---- Demand intensity score ----
    enhanced_df["traffic_score"] = (
        enhanced_df["TrafficConditionNearby"] / 2
    )  # Scaled to 0–1
    enhanced_df["vehicle_weight"] = enhanced_df["VehicleType"].map(
        {1: 0.2, 2: 0.4, 3: 1.0, 4: 1.5}
    )
    # Importance: A combined metric for estimating overall stress on a parking lot.
    # ---- Market density: number of records per lot ----
    lot_density = enhanced_df["SystemCodeNumber"].value_counts()
    enhanced_df["market_density"] = enhanced_df["SystemCodeNumber"].map(lot_density)
    # Importance: Reflects lot activity; useful for filtering inactive lots.
    # ---- Fill missing values from rolling computations ----
    enhanced_df = enhanced_df.fillna(method="bfill").fillna(method="ffill")
    return enhanced_df

In [18]:
fea_df = feature_engineering(df)

In [19]:
print(fea_df.info())
fea_df.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 18368 entries, 0 to 18367
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SystemCodeNumber        18368 non-null  object        
 1   Capacity                18368 non-null  int64         
 2   Latitude                18368 non-null  float64       
 3   Longitude               18368 non-null  float64       
 4   Occupancy               18368 non-null  int64         
 5   VehicleType             18368 non-null  int64         
 6   TrafficConditionNearby  18368 non-null  int64         
 7   QueueLength             18368 non-null  int64         
 8   IsSpecialDay            18368 non-null  int64         
 9   Timestamp               18368 non-null  datetime64[ns]
 10  OccupancyRatio          18368 non-null  float64       
 11  hour                    18368 non-null  int32         
 12  day_of_week             18368 non-null  int32      

,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,Timestamp,OccupancyRatio,hour,day_of_week,is_weekend,closest_lot_distance,occupancy_rolling_3h,traffic_score,vehicle_weight,market_density
ID,,,,,,,,,,,,,,,,,,,
0,BHMBCCMKT01,577,26.144536,91.736172,61,3,0,1,0,2016-10-04 07:59:00,0.105719,7,1,0,0.005672,0.105719,0.0,1.0,1312
1,BHMBCCMKT01,577,26.144536,91.736172,64,3,0,1,0,2016-10-04 08:25:00,0.110919,8,1,0,0.005672,0.108319,0.0,1.0,1312
2,BHMBCCMKT01,577,26.144536,91.736172,80,3,0,2,0,2016-10-04 08:59:00,0.138648,8,1,0,0.005672,0.118429,0.0,1.0,1312


In [ ]:
def baseline_linear_model(df, alpha=0.5, base_price=10):
    lin_df = df.copy()
    lin_df["Price_Model1"] = 0.0
    parking_lots = lin_df["SystemCodeNumber"].unique()

    for lot_id in parking_lots:
        lot_mask = lin_df["SystemCodeNumber"] == lot_id
        lot_data = lin_df[lot_mask].copy()
        lot_data = lot_data.sort_values("Timestamp")
        prices = [base_price]

        for i in range(1, len(lot_data)):
            occ_ratio = lot_data.iloc[i]["OccupancyRatio"]
            new_price = prices[-1] + alpha * occ_ratio
            prices.append(new_price)

        lin_df.loc[lot_data.index, "Price_Model1"] = prices

    return lin_df

In [ ]:
def baseline_linear_model_daily_reset(df, alpha=0.5, base_price=10):
    lin_df = df.copy()
    lin_df["Price_Model2"] = 0.0
    lin_df["date_only"] = lin_df["Timestamp"].dt.date  # Extract date from Timestamp

    parking_lots = lin_df["SystemCodeNumber"].unique()

    for lot_id in parking_lots:
        lot_data = lin_df[lin_df["SystemCodeNumber"] == lot_id]
        unique_dates = lot_data["date_only"].unique()

        for date in unique_dates:
            day_data = lot_data[lot_data["date_only"] == date].copy()
            day_data = day_data.sort_values("Timestamp")
            prices = [base_price]

            for i in range(1, len(day_data)):
                occ_ratio = day_data.iloc[i]["OccupancyRatio"]
                new_price = prices[-1] + alpha * occ_ratio
                new_price = max(0.5 * base_price, min(2 * base_price, new_price))
                prices.append(new_price)

            lin_df.loc[day_data.index, "Price_Model2"] = prices

    lin_df.drop(columns=["date_only"], inplace=True)
    return lin_df

In [23]:
fea_df = baseline_linear_model(fea_df)
fea_df = baseline_linear_model_daily_reset(fea_df)

In [24]:
print(fea_df.columns)
fea_df.head(3)

Index(['SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy',
       'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay',
       'Timestamp', 'OccupancyRatio', 'hour', 'day_of_week', 'is_weekend',
       'closest_lot_distance', 'occupancy_rolling_3h', 'traffic_score',
       'vehicle_weight', 'market_density', 'Price_Model1', 'Price_Model2'],
      dtype='object')


,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,Timestamp,...,hour,day_of_week,is_weekend,closest_lot_distance,occupancy_rolling_3h,traffic_score,vehicle_weight,market_density,Price_Model1,Price_Model2
ID,,,,,,,,,,,,,,,,,,,,,
0,BHMBCCMKT01,577,26.144536,91.736172,61,3,0,1,0,2016-10-04 07:59:00,...,7,1,0,0.005672,0.105719,0.0,1.0,1312,10.000000,10.000000
1,BHMBCCMKT01,577,26.144536,91.736172,64,3,0,1,0,2016-10-04 08:25:00,...,8,1,0,0.005672,0.108319,0.0,1.0,1312,10.055459,10.055459
2,BHMBCCMKT01,577,26.144536,91.736172,80,3,0,2,0,2016-10-04 08:59:00,...,8,1,0,0.005672,0.118429,0.0,1.0,1312,10.124783,10.124783


In [ ]:
def demand_based_model(df, base_price=10, lam=0.4):
    dem_df = df.copy()
    # Vehicle weight mapping
    vehicle_weight_map = {1: 0.2, 2: 0.4, 3: 1.0, 4: 1.5}
    dem_df["vehicle_weight"] = dem_df["VehicleType"].map(vehicle_weight_map)
    # Normalize QueueLength
    max_queue = dem_df["QueueLength"].max()
    dem_df["queue_score"] = dem_df["QueueLength"] / max_queue if max_queue != 0 else 0
    # Demand function: α·OccupancyRatio + β·QueueScore − γ·Traffic + δ·SpecialDay + ε·VehicleWeight
    α, β, γ, δ, ε = 1.0, 0.5, 0.4, 1.0, 0.7
    dem_df["RawDemand"] = (
        α * dem_df["OccupancyRatio"]
        + β * dem_df["queue_score"]
        - γ * dem_df["traffic_score"]
        + δ * dem_df["IsSpecialDay"]
        + ε * dem_df["vehicle_weight"]
    )
    # Normalize demand
    dem_df["NormalizedDemand"] = min_max_normalize(dem_df["RawDemand"])
    # Pricing formula
    dem_df["Price_Model3"] = base_price * (1 + lam * dem_df["NormalizedDemand"])
    # Clip prices to [0.5x, 2x]
    dem_df["Price_Model3"] = dem_df["Price_Model3"].clip(
        lower=base_price * 0.5, upper=base_price * 2
    )
    return dem_df

In [26]:
fea_df = demand_based_model(fea_df)

In [ ]:
def competitive_pricing_model(comp_df, base_price=10, distance_threshold_km=0.5):
    comp_df = comp_df.copy()
    comp_df["Price_Model4"] = comp_df["Price_Model3"]  # Start from demand-based

    grouped = comp_df.groupby("Timestamp")
    result_frames = []

    for timestamp, group in grouped:
        group = group.reset_index(drop=True)
        updated_prices = []
        reroute_flags = []
        avg_comp_prices = []
        num_competitors = []

        for i, row in group.iterrows():
            my_price = row["Price_Model3"]
            my_occ = row["OccupancyRatio"]
            my_lat, my_lon = row["Latitude"], row["Longitude"]

            # Identify competitors
            competitors = []
            for j, comp in group.iterrows():
                if i == j:
                    continue
                dist = haversine(my_lat, my_lon, comp["Latitude"], comp["Longitude"])
                if dist <= distance_threshold_km:
                    competitors.append(
                        {"price": comp["Price_Model3"], "occ": comp["OccupancyRatio"]}
                    )

            # Defaults
            updated_price = my_price
            reroute = False
            avg_price = my_price
            n_comps = len(competitors)

            if competitors:
                comp_prices = [c["price"] for c in competitors]
                avg_price = np.mean(comp_prices)
                cheaper_comps = [c for c in competitors if c["price"] < my_price]
                expensive_comps = [c for c in competitors if c["price"] > my_price]
                full_comps = [c for c in competitors if c["occ"] >= 0.9]

                # Competitive logic
                if my_occ >= 0.9 and cheaper_comps:
                    updated_price = my_price * 0.95
                    reroute = True
                elif my_price < avg_price and expensive_comps:
                    updated_price = my_price * 1.05

            # Clip price
            updated_price = np.clip(updated_price, base_price * 0.5, base_price * 2)

            updated_prices.append(updated_price)
            reroute_flags.append(reroute)
            avg_comp_prices.append(avg_price)
            num_competitors.append(n_comps)

        # Update group
        group["Price_Model4"] = updated_prices
        group["Reroute"] = reroute_flags
        group["AvgCompPrice"] = avg_comp_prices
        group["NumCompetitors"] = num_competitors

        result_frames.append(group)

    comp_df_final = pd.concat(result_frames).sort_values("Timestamp")
    return comp_df_final.reset_index(drop=True)

In [28]:
fea_df = competitive_pricing_model(fea_df, distance_threshold_km=2)

In [ ]:
print(
    fea_df.isnull().sum().sum()
)  # Just checking for any null values after all the model implementations and feature engineering (due to any errors!)
print(fea_df.shape)
print(fea_df.columns)

0
(18368, 29)
Index(['SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy',
       'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay',
       'Timestamp', 'OccupancyRatio', 'hour', 'day_of_week', 'is_weekend',
       'closest_lot_distance', 'occupancy_rolling_3h', 'traffic_score',
       'vehicle_weight', 'market_density', 'Price_Model1', 'Price_Model2',
       'queue_score', 'RawDemand', 'NormalizedDemand', 'Price_Model3',
       'Price_Model4', 'Reroute', 'AvgCompPrice', 'NumCompetitors'],
      dtype='object')


In [30]:
print(fea_df.columns)
print(fea_df.info())

Index(['SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy',
       'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay',
       'Timestamp', 'OccupancyRatio', 'hour', 'day_of_week', 'is_weekend',
       'closest_lot_distance', 'occupancy_rolling_3h', 'traffic_score',
       'vehicle_weight', 'market_density', 'Price_Model1', 'Price_Model2',
       'queue_score', 'RawDemand', 'NormalizedDemand', 'Price_Model3',
       'Price_Model4', 'Reroute', 'AvgCompPrice', 'NumCompetitors'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   SystemCodeNumber        18368 non-null  object        
 1   Capacity                18368 non-null  int64         
 2   Latitude                18368 non-null  float64       
 3   Longitude               18368 non-null  float6

In [31]:
# 1️⃣ Define Pathway schema matching fea_df columns you want to stream
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    OccupancyRatio: float
    VehicleType: float
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int
    Timestamp: pw.DateTimeNaive
    traffic_score: float
    Price_Model2: float
    Price_Model3: float
    AvgCompPrice: float
    Price_Model4: float

In [ ]:
# Subject to feed pandas DataFrame to Pathway
class DFSubject(pw.io.python.ConnectorSubject):
    deletions_enabled = False

    def __init__(self, df):
        super().__init__()
        self.df = df.sort_values("Timestamp")

    def run(self):
        for _, row in self.df.iterrows():
            self.next(**row.to_dict())
        self.close()

In [33]:
# Connect DataFrame as data source
subject = DFSubject(fea_df)
data_with_time = pw.io.python.read(subject, schema=ParkingSchema)

In [ ]:
#  Daily tumbling window per parking lot
daily_window = data_with_time.windowby(
    pw.this.Timestamp,
    instance=pw.this.SystemCodeNumber,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior(),
).reduce(
    SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
    Timestamp=pw.this._pw_window_end,
    OccupancyRatio=pw.reducers.max(pw.this.OccupancyRatio),
    Capacity=pw.reducers.max(pw.this.Capacity),
    Latitude=pw.reducers.max(pw.this.Latitude),
    Longitude=pw.reducers.max(pw.this.Longitude),
    VehicleType=pw.reducers.max(pw.this.VehicleType),
    TrafficConditionNearby=pw.reducers.max(pw.this.TrafficConditionNearby),
    QueueLength=pw.reducers.max(pw.this.QueueLength),
    IsSpecialDay=pw.reducers.max(pw.this.IsSpecialDay),
    traffic_score=pw.reducers.max(pw.this.traffic_score),
    Price_Model2=pw.reducers.max(pw.this.Price_Model2),
    Price_Model3=pw.reducers.max(pw.this.Price_Model3),
    AvgCompPrice=pw.reducers.max(pw.this.AvgCompPrice),
    Price_Model4=pw.reducers.max(pw.this.Price_Model4),
)

In [ ]:
# Bokeh price plotter function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pricing Model Comparison Over Time",
        x_axis_type="datetime",
    )
    fig.line(
        "Timestamp",
        "Price_Model2",
        source=source,
        color="red",
        legend_label="Linear Price",
        line_width=2,
    )
    fig.line(
        "Timestamp",
        "Price_Model3",
        source=source,
        color="green",
        legend_label="Demand-Based Price",
        line_width=2,
    )
    fig.line(
        "Timestamp",
        "AvgCompPrice",
        source=source,
        color="orange",
        legend_label="Competitor Avg Price",
        line_width=2,
        line_dash="dashed",
    )
    fig.line(
        "Timestamp",
        "Price_Model4",
        source=source,
        color="navy",
        legend_label="Final Competitive Price",
        line_width=2,
    )

    fig.scatter(
        "Timestamp",
        "Price_Model4",
        source=source,
        size=5,
        color="navy",
        legend_label="Final Price Points",
    )

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    fig.legend.location = "top_left"
    fig.grid.grid_line_alpha = 0.3

    return fig

In [36]:
fea_df["SystemCodeNumber"].unique()

array(['BHMBCCMKT01', 'Shopping', 'Others-CCCPS98', 'Others-CCCPS8',
       'Others-CCCPS135a', 'Others-CCCPS119a', 'Others-CCCPS105a',
       'Others-CCCPS202', 'BHMNCPNST01', 'BHMNCPHST01', 'BHMMBMMBX01',
       'BHMEURBRD01', 'BHMBCCTHL01', 'Broad Street'], dtype=object)

In [44]:
# One interactive plot per parking lot
unique_lots = [
    "BHMBCCMKT01",
    "Shopping",
    "Others-CCCPS98",
    "Others-CCCPS8",
    "Others-CCCPS135a",
    "Others-CCCPS119a",
    "Others-CCCPS105a",
    "Others-CCCPS202",
    "BHMNCPNST01",
    "BHMNCPHST01",
    "BHMMBMMBX01",
    "BHMEURBRD01",
    "BHMBCCTHL01",
    "Broad Street",
]
panels = []

for lot_id in unique_lots:
    lot_stream = daily_window.filter(pw.this.SystemCodeNumber == lot_id)
    viz = lot_stream.plot(price_plotter, sorting_col="Timestamp")
    title = pn.pane.Markdown(f"### Parking Lot: `{lot_id}`")
    # Add title for each parking lot
    # Append both title and plot together in a vertical layout
    panels.append(pn.Column(title, viz))

pn.extension()
pn.Column(*panels).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'9efbecd1-fdb9-4c76-a4c4-ec6aad1204e0': {'version…

In [ ]:
# Start Pathway streaming engine
# %%capture --no-display
pw.run()

Output()

### Another Method For Using the panel


In [ ]:
class DFSubject1(pw.io.python.ConnectorSubject):
    deletions_enabled = False

    def __init__(self, df):
        super().__init__()
        self.df = df.sort_values("Timestamp")

    def run(self):
        for _, row in self.df.iterrows():
            self.next(
                SystemCodeNumber=row["SystemCodeNumber"],
                Capacity=int(row["Capacity"]),
                Latitude=float(row["Latitude"]),
                Longitude=float(row["Longitude"]),
                Occupancy=int(row["Occupancy"]),
                VehicleType=float(row["VehicleType"]),
                TrafficConditionNearby=float(row["TrafficConditionNearby"]),
                QueueLength=int(row["QueueLength"]),
                IsSpecialDay=int(row["IsSpecialDay"]),
                Timestamp=row["Timestamp"],
                closest_lot_distance=float(row["closest_lot_distance"]),
                occupancy_rolling_3h=float(row["occupancy_rolling_3h"]),
                Price_Model1=float(row["Price_Model1"]),
                Price_Model2=float(row["Price_Model2"]),
                Price_Model3=float(row["Price_Model3"]),
                Price_Model4=float(row["Price_Model4"]),
                Reroute=bool(row["Reroute"]),
                AvgCompPrice=float(row["AvgCompPrice"]),
                NumCompetitors=int(row["NumCompetitors"]),
            )
        self.close()

In [ ]:
def real_time_simulation(df):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    subject = DFSubject1(df)
    stream = pw.io.python.read(subject, schema=ParkingSchema)
    # Pass-through or add any transformations here
    result = stream.select(
        SystemCodeNumber=stream.SystemCodeNumber,
        Timestamp=stream.Timestamp,
        Price_Model1=stream.Price_Model1,
        Price_Model2=stream.Price_Model2,
        Price_Model3=stream.Price_Model3,
        Price_Model4=stream.Price_Model4,
        Reroute=stream.Reroute,
        AvgCompPrice=stream.AvgCompPrice,
        NumCompetitors=stream.NumCompetitors,
    )
    pw.debug.compute_and_print(result)

In [ ]:
def plot_real_time(df, lot_id="Shopping", price_col="Price_Model4"):
    df_lot = df[df["SystemCodeNumber"] == lot_id].sort_values("Timestamp")
    source = ColumnDataSource(data={"x": df_lot["Timestamp"], "y": df_lot[price_col]})
    p = figure(
        x_axis_type="datetime",
        title=f"Real-Time Price Plot: {lot_id} ({price_col})",
        width=500,
        height=300,
    )
    p.line("x", "y", source=source, line_width=2, color="navy", legend_label=price_col)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.legend.location = "top_left"
    p.grid.grid_line_alpha = 0.3
    return p

In [ ]:
def plot_competitive_pricing(df, lot_id="Shopping"):
    df_lot = df[df["SystemCodeNumber"] == lot_id].sort_values("Timestamp")

    source = ColumnDataSource(
        data={
            "x": df_lot["Timestamp"],
            "DemandBasedPrice": df_lot["Price_Model3"],
            "CompetitorAvgPrice": df_lot["AvgCompPrice"],
            "FinalPrice": df_lot["Price_Model4"],
        }
    )

    p = figure(
        x_axis_type="datetime",
        title=f"📈 Competitive Pricing Breakdown: {lot_id}",
        width=500,
        height=300,
    )

    p.line(
        "x",
        "DemandBasedPrice",
        source=source,
        color=Category10[3][0],
        legend_label="Demand-Based (Model 3)",
        line_width=2,
    )
    p.line(
        "x",
        "CompetitorAvgPrice",
        source=source,
        color=Category10[3][1],
        legend_label="Avg Competitor Price",
        line_width=2,
        line_dash="dashed",
    )
    p.line(
        "x",
        "FinalPrice",
        source=source,
        color=Category10[3][2],
        legend_label="Final Adjusted Price (Model 4)",
        line_width=2,
    )

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    p.legend.location = "top_left"
    p.grid.grid_line_alpha = 0.3

    return p

In [ ]:
park_lots = [
    "BHMBCCMKT01",
    "Shopping",
    "Others-CCCPS98",
    "Others-CCCPS8",
    "Others-CCCPS135a",
    "Others-CCCPS119a",
    "Others-CCCPS105a",
    "Others-CCCPS202",
    "BHMNCPNST01",
    "BHMNCPHST01",
    "BHMMBMMBX01",
    "BHMEURBRD01",
    "BHMBCCTHL01",
    "Broad Street",
]
priceModels = ["Price_Model1", "Price_Model2", "Price_Model3", "Price_Model4"]
# Price_Model1 : Just a simple linear model, base price = 10 dollar, without daily reset
# Price_Model2 : Just a simple linear model, base price = 10 dollar, with daily reset
# Price_Model3 : Just a demand based model, base price = 10 dollar
# Price_Model4 : Just a Competitive model, base price = 10 dollar
# Collect panels in a list
panels = []
for park_lot in park_lots:
    for price_model in priceModels:
        if price_model == "Price_Model4":
            panel = plot_competitive_pricing(fea_df, lot_id=park_lot)
        else:
            panel = plot_real_time(fea_df, lot_id=park_lot, price_col=price_model)
        panels.append(panel)
# Arrange panels in a grid with 2 columns
rows = [panels[i : i + 2] for i in range(0, len(panels), 2)]
grid = gridplot(children=rows)
# Show the entire grid
show(grid)